In [1]:
import pandas as pd
import re

In [2]:
df_radiology = pd.read_csv("radiology.csv")

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
df_radiology.head(5)

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,"EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION: ___ with new onset ascites // eval for infection\n\nTECHNIQUE: Chest PA and lateral\n\nCOMPARISON: None.\n\nFINDINGS: \n\nThere is no focal consolidation, pleural effusion or pneumothorax. Bilateral\nnodular opacities that most likely represent nipple shadows. The\ncardiomediastinal silhouette is normal. Clips project over the left lung,\npotentially within the breast. The imaged upper abdomen is unremarkable.\nChronic deformity of the posterior left sixth and seventh ribs are noted.\n\nIMPRESSION: \n\nNo acute cardiopulmonary process.\n"
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN)\n\nINDICATION: ___ year-old female with cirrhosis, jaundice.\n\nTECHNIQUE: Grey scale and color Doppler ultrasound images of the abdomen were\nobtained.\n\nCOMPARISON: None.\n\nFINDINGS: \n\nLIVER: The liver is coarsened and nodular in echotexture. There is no focal\nliver mass. Main portal vein and its major branches are patent with normal\nhepatopetal flow. The main hepatic artery shows normal arterial waveform. \nThere is a small amount of ascites.\n\nBILE DUCTS: There is no intrahepatic biliary dilation. The CBD measures 4 mm.\n\nGALLBLADDER: The gallbladder is contracted with a shadowing gallstone.\n\nPANCREAS: Imaged portion of the pancreas appears within normal limits, without\nmasses or pancreatic ductal dilation, with portions of the pancreatic tail\nobscured by overlying bowel gas.\n\nSPLEEN: Normal echogenicity, measuring 13.5 cm.\n\nKIDNEYS: The right kidney measures 12.1 cm. The left kidney measures 13.4 cm.\nNormal cortical echogenicity and corticomedullary differentiation is seen\nbilaterally.\nRETROPERITONEUM: Visualized portions of aorta and IVC are within normal\nlimits.\n\nIMPRESSION: \n\n1. Nodular appearance of the liver compatible with cirrhosis. Signs of portal\nhypertension including small amount of ascites and splenomegaly.\n2. Cholelithiasis.\n3. Patent portal veins with normal hepatopetal flow.\n"
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD,\nbioplar, PTSD, presented from OSH ED with worsening abd distension over past\nweek. // SBP\n\nTECHNIQUE: Ultrasound guided diagnostic and therapeutic paracentesis\n\nCOMPARISON: Abdominal ultrasound ___\n\nFINDINGS: \n\nLimited grayscale ultrasound imaging of the abdomen demonstrated\nmoderateascites. A suitable target in the deepest pocket in the right lower\nquadrant was selected for paracentesis.\n\nPROCEDURE: The procedure, risks, benefits and alternatives were discussed\nwith the patient and written informed consent was obtained.\n\nA preprocedure time-out was performed discussing the planned procedure,\nconfirming the patient's identity with 3 identifiers, and reviewing a\nchecklist per ___ protocol.\n\nUnder ultrasound guidance, an entrance site was selected and the skin was\nprepped and draped in the usual sterile fashion. 1% lidocaine was instilled\nfor local anesthesia.\n\nA 5 ___ catheter was advanced into the largest fluid pocket in the right\nlower quadrant and 1.5 L of serosanguinous fluid was removed.\n\nThe patient tolerated the procedure well without immediate complication.\nEstimated blood loss was minimal. A sample of the fluid was sent to the lab\nas requested.\n\nDr. ___ attending radiologist, was present throughout the critical\nportions of the procedure.\n\nIMPRESSION: \n\nSuccessful uncomplicated ultrasound guided diagnostic and therapeutic\nparacentesis yielding 1.5 L of serosanguineous fluid from the right lower\nquadrant. Sample was sent to the lab as requested.\n"
3,10000032-RR-18,10000032,NaN,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,"EXAMINATION: Ultraso